### Podatkovno rudarjenje, 1. domača naloga, 9. 3. 2018

# Priprava podatkov, osnovne statistike in vizualizacija

Milosh Kostadinovski

Neizogiben del vsakega projekta na področju podatkovnega rudarjenja je iskanje,
urejanje in priprava podatkov. V tej nalogi boste spoznali primer podatkovne
zbirke in uporabili postopke za pretvorbo podatkov v ustrezno obliko ter pregled in prikaz
osnovnih statistik.

### Oddaja
Zapišite kodo in odgovore v spodnje celice. Tako pripravljen notebook shranite v vaš repozitorij za domače naloge na *github*. V učilnici oddajte le povezavo do notebooka v vašem repozitoriju, n.pr.: https://github.com/vašeuporabniškoime/PR18DNvz/dn1/poročilo.ipynb.

Za bolj podrobna navodila, glejte razdelek "Domače naloge" v [spletni učilnici predmeta](https://ucilnica.fri.uni-lj.si/course/view.php?id=37).

## Podatki

V nalogi boste pregledali in pripravili podatke gledanosti Hollywoodskih filmov
zbirke [MovieLens](https://grouplens.org/datasets/movielens/) v obdobju **1995-2016**. Podatke naložite iz [spletne učilnice](https://ucilnica.fri.uni-lj.si/mod/resource/view.php?id=19230).

Iste podatke boste uporabili v vseh domačih nalogah, zato jih dodobra spoznajte. Gre za podatkovno zbirko za
vrednotenje priporočilnih sistemov, ki vsebuje gledalce ter njihove ocene za posamezni film na lestvici 1 do 5.  
Poleg osnovne matrike uporabnikov in ocen vsebuje še dodatne podatke o filmih (npr. žanr, datum, oznake,
igralci).

Podatkovna zbirka vsebuje naslednje datoteke:

* ratings.csv: podatki o uporabnikih in ocenah,
* movies.csv: podatki o žanrih filmov,
* cast.csv: podatki o igralcih,
* tags.csv: podatki o oznakah (ang. \emph{tags}),
* links.csv: povezave na sorodne podatkovne zbirke.


Pred pričetkom reševanja naloge si dobro oglejte podatke in datoteko **README.txt**. Podrobnosti o zbirki lahko preberete na [spletni strani](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

Pripravite metode za nalaganje podatkov v ustrezne podatkovne strukture. Te vam bodo prišle
prav tudi pri nadaljnjih nalogah.
Bodite pozorni na velikost podatkov.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [4]:
movies = pd.read_csv('ml-latest-small/movies.csv')

## Vprašanja

Glavni namen podatkovnega rudarjenja je *odkrivanje znanj iz podatkov*,
torej odgovarjanje na vprašanja z uporabo računskih postopkov.

Z uporabo principov, ki ste jih spoznali na vajah in predavanjih, odgovorite na
spodnja vprašanja. Pri vsakem vprašanju dobro premislite, na kakšen način boste
najbolje podali, prikazali oz. utemeljili odgovor. Bistven del so odgovori na
vprašanja in ne implementacija vaše rešitve.

### 1. vprašanje (15%):
Kateri filmi so v povprečju najbolje ocenjeni?  Pripravite seznam
filmov ter njihovih povprečnih ocen in izpišite po 10 filmov z vrha seznama.
Opazite pri takem ocenjevanju kakšno težavo? Kako bi jo lahko rešili? Kakšni so
rezultati tedaj?

In [5]:
def top10_movies(ratings):
    ratings_mean = ratings.groupby(['movieId'], as_index = False).agg({'rating': [np.mean]})     # group by movies by their id and calculate movie rating mean
    ratings_mean.columns = ["movieId", "rating_mean"]                                            # set column names in new df

    ratings_mean = ratings_mean.sort_values(by="rating_mean", ascending=[False])                 # sort descending by rating
    top10_movies = ratings_mean[:10]                                                             # get 10 first movies (top10)
    top10_movieIds = ratings_mean[:10]["movieId"]                                                # get top10 movie ids
    top10_movieNames = movies[movies["movieId"].isin(top10_movieIds)]                            # get top10 movie names
    
    result = pd.merge(top10_movies, top10_movieNames, on='movieId')                             # join movies and their names by movieID
    return result[["title", "rating_mean"]] 

top10 = top10_movies(ratings)
top10

,title,rating_mean
0,The Beatles: Eight Days a Week - The Touring Y...,5.0
1,Padre padrone (1977),5.0
2,Amazing Grace (2006),5.0
3,Woman on the Beach (Haebyeonui yeoin) (2006),5.0
4,O Lucky Man! (1973),5.0
5,Dorian Blues (2004),5.0
6,My Best Friend (Mon meilleur ami) (2006),5.0
7,"Edge of Heaven, The (Auf der anderen Seite) (2...",5.0
8,Drained (O cheiro do Ralo) (2006),5.0
9,To the Left of the Father (Lavoura Arcaica) (2...,5.0


**Odgovor**: Težava je v temu, da nekatere filme je ocenil le en uporabnik, nekatere pa več uporabnikov. Sistem povprečja ocen filmov ni zanesljiv. Boljsa rešitev bi bila, da upoštevamo samo tiste filme, ki jih je ocenilo več kot n uporabnikov, kjer je n povprečno število ocen.

In [6]:
ratings_count = ratings.groupby(['movieId'], as_index = False).agg({'rating': [np.size]}) # get number of votes per movie
ratings_count.columns = ["movieId", "number_of_votes"]                                    # set column names in new df
n = int(ratings_count["number_of_votes"].mean())                                          # average number of votes per movie
movie_ids = ratings_count[ratings_count["number_of_votes"] > n]                           # get movies which have more than n votes
movie_ids = movie_ids["movieId"]
movie_ratings = ratings[ratings["movieId"].isin(movie_ids)]

In [7]:
top10 = top10_movies(movie_ratings)
top10

,title,rating_mean
0,Inherit the Wind (1960),4.541667
1,"Godfather, The (1972)",4.487500
2,"Shawshank Redemption, The (1994)",4.487138
3,Tom Jones (1963),4.458333
4,On the Waterfront (1954),4.448276
5,When We Were Kings (1996),4.437500
6,All About Eve (1950),4.434211
7,Ran (1985),4.423077
8,"African Queen, The (1951)",4.420000
9,Mister Roberts (1955),4.411765


### 2. vprašanje (15%):
Posamezni film pripada enemu ali več žanrom. 
Koliko je vseh žanrov? Prikaži porazdelitev žanrov z uporabo ustrezne
vizualizacije.

In [8]:
# gets all genres and puts them in a set (to get unique values)
all_genres = set()
for e in movies["genres"]:
    e = e.split("|")
    e = set(e)
    all_genres |= e

all_genres.remove("(no genres listed)")

In [9]:
print("Vsi žanri: ")
pd.Series(list(all_genres))

Vsi žanri: 


0         Musical
1         Mystery
2          Horror
3         Romance
4       Adventure
5            IMAX
6       Animation
7          Comedy
8             War
9       Film-Noir
10       Children
11         Sci-Fi
12        Western
13         Action
14          Drama
15        Fantasy
16       Thriller
17    Documentary
18          Crime
dtype: object

In [10]:
print("Število žanrov:", len(all_genres))

Število žanrov: 19


In [11]:
# puts all genres in a list
plot_data = list()
for e in movies["genres"]:
    e = e.split("|")
    if "(no genres listed)" in e:
        e.remove("(no genres listed)")
    plot_data += e

In [12]:
# sorts by number of movies
labels, counts = np.unique(plot_data, return_counts=True)

temp = zip(labels, counts)
temp = sorted(temp, key = lambda t: t[1], reverse=True)
labels, counts = [list(c) for c in zip(*temp)]

In [13]:
# draws the plot
data = [go.Bar(
             x=[str(x) for x in labels],
             y=[int(x) for x in counts]
     )]

layout = go.Layout(
    title="Graf števila filmov na žanr",
    xaxis=dict(
        title="Žanri"
    ), 
    yaxis=dict(
        title="Število filmov"
    )
)
fig = go.Figure(data=data, layout=layout)                         # add image='png' to export image
iplot(fig, filename='pandas-bar-chart', show_link=True)

### 3. vprašanje (20%):
Število ocen (ogledov) se za posamezni film razlikuje. Ali
obstaja povezava med gledanostjo in povprečno oceno filma? Opišite postopek, ki
ste ga uporabili pri odgovarjanju na vprašanje.

In [14]:
movies_count_per_rating = ratings.groupby(["movieId"]).agg({"rating": [np.size, np.mean]})
movies_count_per_rating.columns = ["number_of_votes", "rating_avg"]
number_of_votes = movies_count_per_rating["number_of_votes"]
rating_avg = movies_count_per_rating["rating_avg"]

In [15]:
# draws the plot
data = [go.Scatter(
             x=[str(x) for x in rating_avg],
             y=[int(x) for x in number_of_votes],
             mode="markers"
     )]
layout = go.Layout(
    title="Graf povezave med gledanostjo filmov in povprečno oceno",
    xaxis=dict(
        title="Povprečna ocena filma"
    ), 
    yaxis=dict(
        title="Število ogledov"
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='pandas-bar-chart', show_link=True)

***Odgovor***: Z grafa lahko razberemo, da najbolj gledani film imajo oceno med 3,5 in 4,5. 

In [16]:
# Pearson correlation coefficient
print("Pearsonov koeficient korelacijent:", movies_count_per_rating.corr()["rating_avg"][0])

Pearsonov koeficient korelacijent: 0.13082726621823274


***Odgovor***: Z računanjem Pearsonovega koeficienta korelacije lahko ugotovimo, da obstaja neznatna pozitivna povezanost med atributoma, ker je koeficient prenizek.

### 4. vprašanje (30%):
Vsaka ocena je bila vnešena na določen datum (stolpec
*timestamp*).  Ali se popularnost posameznih filmov s časom spreminja?
Problem reši tako, da za dani film ocene razporediš po času ter v vsaki časovni
točki izračunaš povprečje za zadnjih 30, 50, ali 100 ocen. Nariši graf, kako se
ocena spreminja in ga prikaži za dva zanimiva primera filmov.

In [17]:
from datetime import datetime

In [18]:
def get_movie_rating_data(movie_id, n):
    movie_ratings = ratings.groupby("movieId").get_group(movie_id)
    movie_ratings = movie_ratings.sort_values(by="timestamp")
    # converts timestamp to datetime
    dates = [datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M') for t in movie_ratings["timestamp"]]
    dates = dates[n:]
    movie_ratings_avg = []
    for i in range(n, len(dates)):
        movie_ratings_avg += [movie_ratings["rating"].iloc[i - 30: i].mean()]
    return dates, movie_ratings_avg

In [19]:
def draw_movie_rating_graph(dates, movie_ratings_avg):
    # draws the plot
    data = [go.Scatter(
                 x=dates,
                 y=movie_ratings_avg
         )]

    layout = go.Layout(
        title="Graf spremembe povprečne ocene s časom",
        xaxis=dict(
            title="Čas"
        ), 
        yaxis=dict(
            title="Povprečna ocena"
        )
    )
    #iplot(data, filename='pandas-bar-chart', show_link=True, image='png')
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='pandas-bar-chart', show_link=True)

In [20]:
# movie id = 1206, take last 30 votings and calculate avg. If graph doesn't show, lower second argument.
dates, movie_ratings_avg = get_movie_rating_data(6, 30)
draw_movie_rating_graph(dates, movie_ratings_avg)

In [21]:
# movie id = 1206, take last 30 votings and calculate avg. If graph doesn't show, lower second argument.
dates, movie_ratings_avg = get_movie_rating_data(1206, 30)
draw_movie_rating_graph(dates, movie_ratings_avg)

**Odgovor**: Popularnost filma se s časom spreminja in ne sledi nobenemu določenemu vzorcu.

### 5. vprašanje (20%):
Kako bi ocenili popularnost posameznih igralcev? Opišite postopek
ocenitve ter izpišite 10 najbolj popularnih igralcev.

In [22]:
cast = pd.read_csv('ml-latest-small/cast.csv')

In [23]:
from collections import defaultdict
from collections import OrderedDict

In [24]:
numb_of_movies = defaultdict(int)

for actors in cast["cast"]:
    if type(actors) is float:
        continue
    for e in actors.split("|"):
        numb_of_movies[e] += 1

numb_of_movies.pop("")
print("Najbolj popularni igralci: ")
sorted(numb_of_movies, key=numb_of_movies.get, reverse=True)[:11]

Najbolj popularni igralci: 


['Robert De Niro',
 'Bruce Willis',
 'Michael Caine',
 'Morgan Freeman',
 'Christopher Walken',
 'Nicolas Cage',
 'Richard Jenkins',
 'Steve Buscemi',
 'John Goodman',
 'Alec Baldwin',
 'Bill Murray']

**Odgovor**: Najbolj popularne igralce sem dobil tako, da sem najprej dobil število filmov v katerih je vsak igralec igral. Nato sem uredil neznam po številu filmov padajoče in izpisal prvih 10 vrednosti.

### bonus vprašanje (5%):

Kateri je tvoj najljubši film? Zakaj?

**Odgovor**: Moj najbuljši film je Stalker režiserja Andreja Tarkovskega. V filmu so mi vseč dolgi prizori s počasnim, dobrim gibanjem kamere. S tem se režiser izogiba hitrega pripovedovanja in doseže zanimivejši opis pripovedi. Film je posnet po romanu in ohranja znanstvenofantastičnih lastnosti romana, ampak je poudarek na filozofiji in psihologiji človeka. Te tematike so mi osebno zelo zanimive. Film je zame prava mojstrovina.

## Zapiski
Za nalaganje podatkov lahko uporabite vgrajen modul `csv`. Mapa s podatki `ml-latest-small` se v tem primeru mora nahajati v isti mapi kot notebook.

In [25]:
from csv import DictReader

reader = DictReader(open('ml-latest-small/ratings.csv', 'rt', encoding='utf-8'))
for row in reader:
    user = row["userId"]
    movie = row["movieId"]
    rating = row["rating"]
    timestamp = row["timestamp"]

Podatki v zadnji vrstici datoteke:

In [26]:
user, movie, rating, timestamp

('671', '6565', '3.5', '1074784724')

Pretvorba časovnega formata (*Unix time*). Kode za oblikovanje so navedene v dokumentaciji modula [`datetime`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

In [27]:
from datetime import datetime

t = 1217897793 # Unix-time
ts = datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M')
ts

'2008-08-05 02:56'